**INSTALLATION**

In [ ]:
%cd ./DNN-Prophecy
!pwd
!ls -lt ./dataset_models/acasxu/

In [ ]:
import math
import io
import os
from collections import namedtuple
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import tree
from tqdm import tqdm
import operator
import pandas as pd
#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
print(tf.__version__)

In [ ]:
!pip install -r requirements.txt

In [ ]:
#! pip install onnx
#! pip install onnx-tf

#! pip install pybind11
#! pip install cmake

import torch
import onnx
#!pip install onnxruntime
import onnxruntime


In [ ]:
#!pip install maraboupy

In [ ]:
!pip install onnx2keras

In [ ]:
import shutil
marabou_path = '/usr/local/lib/python3.11/dist-packages/maraboupy/'
source_file1 = './dataset_models/MarabouNetworkONNX.py'
source_file2 = './dataset_models/ONNXParser.py'
    #destination_file1 = '/usr/local/lib/python3.11/dist-packages/maraboupy/MarabouNetworkONNX.py'
destination_file1 = marabou_path + '/MarabouNetworkONNX.py'
destination_file2 = marabou_path + '/parsers/ONNXParser.py'

shutil.copy(source_file1, destination_file1)
shutil.copy(source_file2, destination_file2)

**GET MODEL AND DATA**

In [ ]:
acas_train = np.empty([384221,5],dtype=float)
acas_train_labels = np.zeros(384221,dtype=int)

def read_inputs_from_file(inputFile):
    global acas_train, acas_train_labels, num
    with open(inputFile) as f:
        lines = f.readlines()
        print(len(lines), "examples")
        acas_train = np.empty([len(lines),5],dtype=float)
        acas_train_labels = np.zeros(len(lines),dtype=int)

        for l in range(len(lines)):
            k = [float(stringIn) for stringIn in lines[l].split(',')]
            if len(k) > 5:
              lab = int(k[5])
              acas_train_labels[l+num] = lab

            count = 0
            for i in range(0,5):
                acas_train[l+num][i] = k[i]

!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/clusterinACAS_0_short.csv -O ./clusterinACAS_0_shrt.csv

num = 0


read_inputs_from_file('./clusterinACAS_0_shrt.csv')



print((acas_train).shape)


train_len = (len(acas_train) * 0.75)
train_data = acas_train[:int(train_len)]
train_labels = acas_train_labels[:int(train_len)]

test_data = acas_train[int(train_len):]
test_labels = acas_train_labels[int(train_len):]

print((train_data).shape)
print((train_labels).shape)
print((test_data).shape)
print((test_labels).shape)




In [ ]:
import onnx
from onnx2keras import onnx_to_keras

onnx_model = onnx.load('./dataset_models/acasxu/ACASXU_experimental_v2a_1_1.onnx')
print("ONNX:")
onnx.checker.check_model(onnx_model)

print(onnx.helper.printable_graph(onnx_model.graph))

In [ ]:
# Convert to Keras model using the custom naming policy
keras_model = onnx_to_keras(onnx_model,['input'])#,name_policy="short")#custom_name_policy)
print("KERAS:")
keras_model.summary()



In [ ]:
if os.path.exists('./dataset_models/acasxu/acasx_onnx_keras.h5'):
  os.remove('./dataset_models/acasxu/acasx_onnx_keras.h5')
# Save Keras model
keras_model.save('./dataset_models/acasxu/acasx_onnx_keras.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow import keras

x_train = np.array(train_data)
y_train = np.eye(5)[train_labels]
y_train1 = tf.argmax(y_train, 1)

x_test = np.array(test_data)
y_test = np.eye(5)[test_labels]
y_test1 = tf.argmax(y_test, 1)

print(np.shape(x_train))
print(np.shape(x_test))
print(np.shape(y_train))
print(np.shape(y_test))


np.save('./x_train_npy.npy', x_train)
np.save('./x_test_npy.npy', x_test)
np.save('./y_train_npy.npy', y_train1)
np.save('./y_test_npy.npy', y_test1)



model=keras.models.load_model('./dataset_models/acasxu/acasx_onnx_keras.h5')
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

x_train_rshape = x_train.reshape(-1, 1, 1, 5)
x_test_rshape = x_test.reshape(-1, 1, 1, 5)
print(np.shape(x_train_rshape))
print(np.shape(x_test_rshape))
np.save('./x_train_rshape.npy', x_train_rshape)
np.save('./x_test_rshape.npy', x_test_rshape)

y_predict = model.predict(x_train_rshape)
y_predict1 = tf.argmin(y_predict, 1)
np.save('./y_predict_npy.npy', y_predict1)

correct_prediction1 = tf.equal(tf.argmin(y_predict, 1), tf.argmax(y_train, 1))
accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))
print(accuracy1.numpy())

y_predict_test = model.predict(x_test_rshape)
y_predict1_test = tf.argmin(y_predict_test, 1)
np.save('./y_predict_test_npy.npy', y_predict1_test)

(288165, 5)
(96056, 5)
(288165, 5)
(96056, 5)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 5), dtype=float32) Tensor("Placeholder_1:0", shape=(5,), dtype=float32)
(288165, 1, 1, 5)
(96056, 1, 1, 5)
Tensor("model/Operation_1_MatMul/MatMul:0", shape=(None, 50), dtype=float32) Tensor("model/Operation_1_Add_const2/Const:0", shape=(50,), dtype=f

**INVOKE PROPHECY TO EXTRACT RULES**

**PARAMETERS: rules per label (based on given labels array), rules in terms of neuron values, TOP rules (highest train recall), just from layer "rule_3".**

In [ ]:
import os
import shutil

if os.path.exists('/results/acasxu/rules/'):
  shutil.rmtree('/results/acasxu/rules/')

os.makedirs('/results/acasxu/rules/')

!pwd
!python -m prophecy.main -m './dataset_models/acasxu/acasx_onnx_keras.h5' -wd './results/acasxu/rules/' analyze -tx ./x_train_rshape.npy -ty ./y_predict_npy.npy -vx ./x_test_rshape.npy -vy ./y_predict_test_npy.npy -layer_name 'relu_3' -type 3 -top True

In [ ]:
import pandas as pd
_output_path = "./results/acasxu/rules/ruleset.csv"


print("****** RULES ********")
#df_op = pd.read_csv(_output_path)
#df_op

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]
ruleset

In [ ]:
!ls -lt '/usr/local/lib/python3.11/dist-packages/maraboupy/'

In [ ]:
!python -m prophecy.main -m './dataset_models/acasxu/acasx_onnx_keras.h5' -wd './results/acasxu/rules/' prove -tx ./x_train_rshape.npy -ty ./y_predict_npy.npy  -mp '/usr/local/lib/python3.11/dist-packages/maraboupy/' -onx './dataset_models/acasxu/ACASXU_experimental_v2a_1_1.onnx' -onx_map './dataset_models/acasxu/acasx_onnx_map.csv' -label 0 -pred True -min_const True